<a href="https://colab.research.google.com/github/harjeet88/agentic_ai/blob/main/autogen/gemini/gemini_as_custom_assitant_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
GEMINI_API_KEY=GOOGLE_API_KEY

In [3]:
import os
os.environ['GOOGLE_API_KEY']=GOOGLE_API_KEY
os.environ['GEMINI_API_KEY']=GEMINI_API_KEY

In [5]:
%pip install -U -q autogen autogen_ext[openai] google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 838.3/838.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.4/306.4 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.8 MB/s eta 0:00:00


In [13]:
import autogen
import google.generativeai as genai
from autogen import AssistantAgent, UserProxyAgent
from autogen_ext.models import *

In [7]:
genai.configure(api_key=GEMINI_API_KEY)

In [18]:
gemini_flash="gemini-2.0-flash"
gemma_1b="gemma-3-1b-it"
gemma_4b="gemma-3-4b-it"
gemma_12b="gemma-3-12b-it"
gemma_27b="gemma-3-27b-it"
gemma_e4b="gemma-3n-e4b-it"
llama3="llama3.2:1b"

In [19]:
def ask_gemini(prompt, model=gemini_flash):
    response = genai.GenerativeModel(model).generate_content(prompt)
    return response.text

In [20]:
# 3. Create Custom Agent
# 3. Create Custom Agent with Corrected Method Signature
class GeminiAgent(AssistantAgent):
    def __init__(self, name, **kwargs):
        super().__init__(name, **kwargs)

    def generate_reply(self, messages=None, sender=None, **kwargs):
        # Extract conversation history
        if messages is None:
            messages = self.chat_messages[sender]

        # Format for Gemini (simple concatenation)
        prompt = "\n".join([msg["content"] for msg in messages])

        # Get Gemini response
        response = ask_gemini(prompt)
        return response
# 4. Setup Agents
gemini_assistant = GeminiAgent(
    name="Gemini_Expert",
    system_message="You are a helpful assistant powered by Google Gemini",
)

In [21]:
user_proxy = UserProxyAgent(
    name="User_Proxy",
    human_input_mode="NEVER",  # Auto-reply
    max_consecutive_auto_reply=3,
    code_execution_config={"work_dir": "coding"},
)


In [22]:
# 5. Start Conversation
user_proxy.initiate_chat(
    gemini_assistant,
    message="Explain quantum computing like I'm 10 years old"
)

User_Proxy (to Gemini_Expert):

Explain quantum computing like I'm 10 years old

--------------------------------------------------------------------------------
Gemini_Expert (to User_Proxy):

Okay, imagine you have a regular light switch. It can be either ON or OFF, right?

Well, a regular computer uses bits, and a bit is like that light switch: it can be either a 0 (OFF) or a 1 (ON). They use lots and lots of these bits to do amazing things, like play games and search the internet.

Now, imagine a special light switch that's not just ON or OFF.  Imagine it can be BOTH ON and OFF *at the same time*!  That's kind of like a **qubit**, which is what a quantum computer uses instead of regular bits.

Think of it like a spinning coin.  Before it lands, it's sort of heads AND tails at the same time.  A qubit can be a 0, a 1, or a mix of both!  We call this "both at the same time" thing **superposition**. It's like a magic superpower!

Because qubits can be in many states at once, quantum co

ChatResult(chat_id=None, chat_history=[{'content': "Explain quantum computing like I'm 10 years old", 'role': 'assistant', 'name': 'User_Proxy'}, {'content': 'Okay, imagine you have a regular light switch. It can be either ON or OFF, right?\n\nWell, a regular computer uses bits, and a bit is like that light switch: it can be either a 0 (OFF) or a 1 (ON). They use lots and lots of these bits to do amazing things, like play games and search the internet.\n\nNow, imagine a special light switch that\'s not just ON or OFF.  Imagine it can be BOTH ON and OFF *at the same time*!  That\'s kind of like a **qubit**, which is what a quantum computer uses instead of regular bits.\n\nThink of it like a spinning coin.  Before it lands, it\'s sort of heads AND tails at the same time.  A qubit can be a 0, a 1, or a mix of both!  We call this "both at the same time" thing **superposition**. It\'s like a magic superpower!\n\nBecause qubits can be in many states at once, quantum computers can try out lot